<font color='red'>注：此处是文档第226页</font>

写在前面：这个模型在“[09-混合前端的seq2seq模型部署-聊天机器人](../09-%E6%B7%B7%E5%90%88%E5%89%8D%E7%AB%AF%E7%9A%84seq2seq%E6%A8%A1%E5%9E%8B%E9%83%A8%E7%BD%B2-%E8%81%8A%E5%A4%A9%E6%9C%BA%E5%99%A8%E4%BA%BA.ipynb)”中已经实现过一遍了，这次相当于换数据再次实现，所以应该尝试自己实现一下，这里写的模型说明很详细。

# 聊天机器人教程

在本教程中，我们探索一个好玩有趣的循环的序列到序列（sequence-to-sequence）的模型用例。我们将用[Cornell Movie-Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)处的电影剧本来训练一个简单的聊天机器人。

在人工智能研究领域中，对话模型是一个非常热门的话题。聊天机器人可以在各种设置中找到，包括客户服务应用和在线帮助。这些机器人通常 由基于检索的模型提供支持，这些模型的输出是某些形式问题预先定义的响应。在像公司IT服务台这样高度受限制的领域中，这些模型可能足够了， 但是，对于更一般的用例它们还不够健壮。让一台机器与多领域的人进行有意义的对话是一个远未解决的研究问题。最近，深度学习热潮已经允许 强大的生成模型，如谷歌的神经对话模型[Neural Conversational Model](https://arxiv.org/abs/1506.05869)，这标志着向多领域生成对话模型迈出了一大步。 在本教程中，我们将在PyTorch中实现这种模型。

**教程要点**
- 对[Cornell Movie-Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)数据集的加载和预处理
- 用[Luong attention mechanism(s)](https://arxiv.org/abs/1508.04025)实现一个sequence-to-sequence模型
- 使用小批量数据联合训练解码器和编码器模型
- 实现贪婪搜索解码模块
- 与训练好的聊天机器人互动

**大致流程**
- 数据预处理：将文本数据处理成适合格式化处理的格式
- 数据编码
- 编码器：使用RNN的GRU实现编码器
- 注意力模块：用于解码器
- 解码器
- 编码器训练方法
- 解码器训练方法
- 整合编码器与解码器到一个模型中
- 模型使用方法
- 模型评估

## 1.下载数据文件
下载数据文件点击[这里](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)并将其放入到当前目录下的 `data/` 文件夹下。之后我们引入一些必须的包。

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math